<center><img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F450%2F1*CXZ804tKLPy2hiikJbYH3w.png&f=1&nofb=1" width=30% ></center>

# <center> Assignment 4: Image Alignment and Stitching </center>
<center> Computer Vision 1, University of Amsterdam </center>
    


In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

In [ ]:
# Make sure you're using the provided environment!
assert cv2.__version__ == "3.4.2", "You're not using the provided Python environment!"
assert np.__version__ == "1.19.5", "You're not using the provided Python environment!"
assert matplotlib.__version__ == "3.3.4", "You're not using the provided Python environment!"
# Proceed to the next cell if you don't get any error.

In [ ]:
#Adding Plotting paramemters
matplotlib.rcParams['figure.figsize']  = (20.0, 10.0)
matplotlib.rcParams['axes.grid']       = False
matplotlib.rcParams['font.size']       = 30
matplotlib.rcParams['axes.labelsize']  = 0.8*plt.rcParams['font.size']
matplotlib.rcParams['axes.titlesize']  = 0.9*plt.rcParams['font.size']
matplotlib.rcParams['legend.fontsize'] = plt.rcParams['font.size']
matplotlib.rcParams['xtick.labelsize'] = 0.5*plt.rcParams['font.size']
matplotlib.rcParams['ytick.labelsize'] = 0.5*plt.rcParams['font.size']
matplotlib.rcParams['scatter.marker']  = 'o'
matplotlib.rcParams['axes.titlepad']   = 20
matplotlib.rcParams['axes.labelpad']   = 20
matplotlib.rcParams['xtick.major.pad'] ='10'
matplotlib.rcParams['ytick.major.pad'] ='10'

# 1. Image Alignment with SIFT & RANSAC

We build a function that takes two images as
input and computes the affine transformation between them. The overall scheme is as follows:

  1.  Detect interest points in each image.

  2.  Characterize the local appearance of the regions around interest
      points.

  3.  Get the set of supposed matches between region descriptors in each
      image.

  4.  Perform RANSAC to discover the best transformation between images. RANSAC is performed as follows:

  -   Repeat $N$ times:

  -   Pick $P$ matches at random from the total set of matches $T$.

  -   Construct a matrix $A$ and vector $b$ using the $P$ pairs of points and find affine transformation parameters $(m1, m2, m3, m4, t1, t2)$ by solving the equation $Ax = b$. Such equation can be solved using the pseudo-inverse: $x = (A^T A)^{-1} A^T b$, or packages of Numpy in Python.

  - Using the transformation parameters, transform the locations of all $T$ points in image1. If the transformation is correct, they should lie close to their counterparts in image2. Plot the two images side by side with a line connecting the original $T$ points in image1 and transformed $T$ points over image2.
      
  - Count the number of inliers, where inliers are defined as the number of transformed points from image1 that lie within a radius of $10$ pixels of their pair in image2.

  - If this count exceeds the best total so far, we save the transformation parameters and the set of inliers.

  - End repeat.

5. Transform image1 using this final set of transformation parameters. If you display this image, you should find that the pose of the object in the scene should correspond to its pose in image2. To transform the image, we use our own function based on **nearest-neighbor interpolation**. We then compute the affine transformation using the OpenCV built-in function `cv2.warpAffine` and compare the results.<br>

## Question 1

### 1.1 Keypoint matching

The following contains a function that takes two image pairs, and return the keypoint matching between them using the built-in cv2 keypoint and matching functions.


In [ ]:
img1_path = "street1.png" 
img2_path = "street2.png"

# Open images
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# Note: OpenCV uses BGR instead of RGB
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) 
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# Display images
fig, ax = plt.subplots(1,2, figsize=(10,10))
plt.suptitle('Building affine transformations')
ax[0].imshow(img1)
ax[0].axis('off')
ax[0].set_title('Image 1')
ax[1].imshow(img2)
ax[1].axis('off')
ax[1].set_title('Image 2')
txt= r"Figure 1.1: The two images used as a basis to develop the affine transformation"
plt.figtext(0.5, 0.2, txt, wrap=True, horizontalalignment='center', fontsize=15)
plt.show()
plt.close()

In [ ]:
def keypoint_matching(image1, image2):
    """
    Given two input images, find and return the matching keypoints.
    Arguments:
    image1: the first image (in RGB)
    image2: the second image (in RGB)
    Returns: 
    The keypoints of image1, the keypoints of image2 and the matching
    keypoints between the two images
    """

    print('\nFinding matching features...')
  
    
    
    # Initiate the SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    
    # Find the keypoints and descriptors with SIFT
    keypoints_1, des_1 = sift.detectAndCompute(image1,None)
    keypoints_2, des_2 = sift.detectAndCompute(image2,None)
    
    # Draw images of keypoints
    img = cv2.drawKeypoints(image1, keypoints_1, image1, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite('sift_keypoints_1.jpg',img)
    
    img = cv2.drawKeypoints(image2, keypoints_2, image2, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2.imwrite('sift_keypoints_2.jpg',img)
    
    # Create BFMatcher object
    bf = cv2.BFMatcher(crossCheck=True)
    
    # Match Descriptors
    matches = bf.match(des_1,des_2)
    
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)

    
    print("Number of keypoints in img1:        ", len(keypoints_1))
    print("Number of keypoints in img2:        ", len(keypoints_2))
    print("Number of keypoints after matching: ", len(matches), "\n")

    return keypoints_1, keypoints_2, matches

In [ ]:
# Find and match key points
keypoints_1, keypoints_2, matches = keypoint_matching(img1,img2)

### Question 1.2
We take a random subset (with set size set to 10) of all matching points, and plot them on the image. We connect matching pairs with lines. 

In [ ]:
# Extract 10 random matches to plot
random_matches = [matches[i] for i in random.sample(range(0, len(matches)), 10)]

# Plot random selection of key point matches
img3 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, random_matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize = (15,15))
plt.title('10 Randomly selected matches between the two images')
txt= r"Figure 1.2: 10 matches drawn with lines inbetween the corresponding points. The matches are calculated using the built-in keypoint finder and matcher of cv2."
plt.figtext(0.5, 0.2, txt, wrap=True, horizontalalignment='center', fontsize=15)
plt.imshow(img3),plt.show()
cv2.imwrite('sift_keypoints_3.jpg',img3)
    

### Question 1.3
Create a function that performs the RANSAC algorithm as explained above. The function should return the best transformation found. For visualization, show the transformations from image1 to image2 and from image2 to image1.

In [ ]:
# Self-designed functions 'get_grid' and 'affine_transform' for ranasc algorithm.

def get_grid(w, h):
    # Generates HOMOGENEOUS coordinates of an image in a grid-like fashion.
    """
    Arguments:
    w, h: the width and height of the image
    Returns: 
    HOMOGENEOUS coordinates
    """
    dst_y, dst_x = np.indices((h, w))
    dst_grid = np.stack((dst_x.ravel(), dst_y.ravel(), np.ones(dst_y.size)))
    return dst_grid


def affine_transform(img, mat, warp='forward'):
    #Applies the affine transformation defined in mat to the image contained in img.
    #Works in homogeneous coordinates.
    """
    Arguments:
    img: the first/second image (img1 or img2)
    mat: transformation matrix
    warp: forward or inverse warping
    Returns:
    transformed image
    """
    h, w = img.shape[:2]

    # Make grid to from the image coordinates
    grid = get_grid(w, h)
    x_orientation, y_orientation = grid[0], grid[1]
        
    warped_grid = np.round(mat@grid).astype(np.int)
    
    if warp =='forward':
        # Apply transformation on grid with mat
        warped_grid = np.round(mat@grid).astype(np.int)

        # Get grid lines
        x_warped, y_warped = warped_grid[0,:], warped_grid[1,:]


        # Get pixels within image boundary
        indices = np.where((x_warped >= 0) & (x_warped < w) &
                           (x_warped >= 0) & (y_warped < h))

        # Get orientation vectors within boundaries
        src_x, src_y = x_orientation[indices].astype(int), y_orientation[indices].astype(int)
        dst_x, dst_y = x_warped[indices].astype(int), y_warped[indices].astype(int)


        # Map the pixel to new location
        dst_map = np.zeros_like(img)
        dst_map[dst_y, dst_x] = img[src_y,src_x]
        return dst_map

    if warp == 'inverse':
        inv_warped_grid = np.round(np.linalg.inv(mat)@grid).astype(np.int)        
        inv_x_warped, inv_y_warped = inv_warped_grid[0,:], inv_warped_grid[1,:]
        
        # Get pixels within image boundary
        indices = np.where((inv_x_warped >= 0) & (inv_x_warped < w) &
                       (inv_y_warped >= 0) & (inv_y_warped < h))
        
        # Get orientation vectors within boundaries
        
        src_x, src_y = x_orientation[indices].astype(int), y_orientation[indices].astype(int)
        dst_x, dst_y = inv_x_warped[indices].astype(int), inv_y_warped[indices].astype(int)
    
        # Map the pixel to new location
        dst_map = np.zeros_like(img)
        dst_map[src_y,src_x] = img[dst_y, dst_x]
        
        return dst_map

In [ ]:
def ransac(kp1, kp2, matches, N, print_vals = True):
    """
    Arguments:
      kp1: the keypoints of image1, 
      kp2: the keypoints of image2
      matches: the matching keypoints between the two images
      N: number of iterations
    Returns: 
      the best transformation matrix
    """
    # Determine number of randomly selected P values from set T
    P = 3
    best_inliers = 0
    best_matrix = None
    

    
    # Run estimations N-times
    for i in range(N):
        # 1. Pick P random matches from T 
        random_matches = [matches[i] for i in random.sample(range(0, len(matches)), P)]
        
        # Obtain x, y, x_prime, y_prime
        
        A = np.empty((0, 6))
        b = np.empty((0, 1))
        for mat in random_matches: 
            # Get the matching keypoints for each of the images
            img1_idx = mat.queryIdx
            img2_idx = mat.trainIdx

            # Get the coordinates
            x, y = kp1[img1_idx].pt
            x_prime, y_prime = kp2[img2_idx].pt
            A = np.vstack((A, np.array([[x, y, 0, 0, 1, 0], [0, 0, x, y, 0, 1]])))
            b = np.vstack((b, np.array([[x_prime], [y_prime]])))
        
        m1, m2, m3, m4, t1, t2 = (np.linalg.pinv(A)@b).T[0]
        
        # 4. Transform the location of all T points in image 1
        affine_transform_mat = np.array([[m1, m2, t1], [m3, m4, t2], [0, 0, 1]])
        
        temp_inliers = 0
        for match in matches:
            point_index = match.queryIdx
            point2_index = match.trainIdx
            point_x, point_y = kp1[point_index].pt
            homogeneous_point = np.array([[point_x], [point_y], [1]])
            transformed_homogeneous_point = affine_transform_mat@homogeneous_point
            transformed_point = np.array([
                transformed_homogeneous_point[0]/transformed_homogeneous_point[2],
                transformed_homogeneous_point[1]/transformed_homogeneous_point[2]]).T[0]
            
            # 5. Count inliners 
            dist = np.linalg.norm(transformed_point-np.array(kp2[point2_index].pt))
            if dist < 10:
                temp_inliers += 1
        
        # Compare to previous itteration
        if temp_inliers > best_inliers:
            best_inliers = temp_inliers
            best_matrix = affine_transform_mat


    if print_vals:
        print("Total number of matches: ", len(matches))
        print("Inliers found:           ", best_inliers)
        print("Outliers removed:        ", len(matches) - best_inliers)

    return best_matrix, best_inliers

In [ ]:
def visualization(img, best_matrix):
    """
    Arguments:
      img: the first/second image (img1 or img2)
      best_matrix: the best transformation matrix
    """  
    # Visualize/export a comparison: "Forward warping", "Inverse warping", 
    # "OpenCV warping", "Original image". Refer to the output of the next
    # cell to see the expected output.

    # Read the image
    rows, cols = img.shape[:2]

    # Apply the affine transformation using cv2.warpAffine()
    dst_cv2 = cv2.warpAffine(img, best_matrix[:2,:], (cols,rows))
    
    # Calculate forward 
    dst_forward = affine_transform(img1, best_matrix)

    # Calculate forward 
    dst_inverse = affine_transform(img1, best_matrix, warp = 'inverse')

    
    # Display the image
    plt.figure(figsize = (20,20))
    plt.suptitle('Image warping with 3 different methods')
    plt.subplot(141)
    plt.imshow(img)
    plt.title('A.: Original image')
    plt.subplot(142)
    plt.imshow(dst_cv2)
    plt.title('B.: Open CV warping')
    plt.subplot(143)
    plt.imshow(dst_forward)
    plt.title('C.: Forward warping')
    plt.subplot(144)
    plt.imshow(dst_inverse)
    plt.title('D.: Inverse warping')
    txt= r"Figure 1.3: Image warping of image 1 using three different methods."
    plt.figtext(0.5, 0.3, txt, wrap=True, horizontalalignment='center', fontsize=15)
    plt.savefig('wrapping.png')

In [ ]:
N_iterations = 50 # experiment with this value!
best_matrix, inliers = ransac(keypoints_1,keypoints_2,matches, N_iterations)
img1 = cv2.imread(img1_path)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) 
visualization(img1,best_matrix)

### 1.4 Comparing the results

In the above we apply the affine transformation to the original image 1 using three methods: the openCV built-in warping function, and our self implemented forward and inverse warping functions using SIFT and RANSAC. 

When computing the forward warping transformation (Figure 1.3.C) we can clearly observe dark structures, or empty pixel slots, which are not present in the OpenCV implementation or the rotated version of the original image. These effects occur due to aliasing, artifacts due to under-sampling, and holes. It can happen when performing a forward warping that some pixels are not taken into account in the new mapping, as their exact positioning under the warping will be located between the pixels of the warped grid. 

One way to correct for this effect is 'splatting', which is to distribute color among the neighboring pixels. Another wasy to implement approach to prevent this aliasing is to perform an inverse transformation, i.e. to use the inverted version of the parameter matrix A. In this sense, we resample the source image upon the warped grid, using a nearest neighbor approach instead of an interpolation mechanism. As can be observed in figure 1.3.D this methodology is successful in implementing a transformation that is at surface level equivalent to the openCV implementation.

## Question 2
Based on the results, answer the following questions.

<a name="q2.1"></a>
### Question 2.1
How many matches do we need to solve an affine transformation which can be formulated as follows:

$$
\begin{bmatrix}
x'\\y'\end{bmatrix} =
\begin{bmatrix}
m_1 & m_2\\
m_3 & m_4
\end{bmatrix}
\begin{bmatrix}
x\\y\end{bmatrix}+
\begin{bmatrix}
t_1\\t_2\end{bmatrix}
$$

The equation above can be rewritten as:

$$
\begin{bmatrix}
x&y&0&0&1&0\\
0&0&x&y&0&1\end{bmatrix}
\begin{bmatrix}
m_1\\
m_2\\
m_3\\
m_4\\
t_1\\
t_2
\end{bmatrix} =
\begin{bmatrix}
x'\\y'\end{bmatrix}
$$
or, alternatively:
$$
Ax=b, \;
A = \begin{bmatrix}
x&y&0&0&1&0\\
0&0&x&y&0&1\end{bmatrix}, \;
x = \begin{bmatrix}
m_1\\
m_2\\
m_3\\
m_4\\
t_1\\
t_2
\end{bmatrix}, \;
b = \begin{bmatrix}
x'\\y'\end{bmatrix}
$$

**ADD YOUR ANSWER HERE**

### 2.2: The number of iterations in average needed to find good transformation parameters

In the section below we run E (E = 15) experiments of calulacting the max number of inliers after N iterations, allowing N to run from 1 to 50 for each experiment.  The values of E and N are found by systematic trial and error. We then take the average of all experiments $E_i$ and plot them as a function of N.

From looking at Figure 2.2.1 we can observe that the average max number of inliers start to converge around N = 20 to a value of approximatly 800, which appears to be the convergence value for these set of matches. As such, it would be good practice to select a value slightly higher than N = 20 itterations to find good transformation parameters. 

**ADD YOUR ANSWER HERE**

In [ ]:
N_iterations = 50 # experiment with this value!
N_vec = np.arange(1, N_iterations + 1, 1)
E_experiments = 15

Inliers_mat = np.zeros((E_experiments, len(N_vec)))

for i in range(E_experiments):
    for m in N_vec:
        best_matrix, best_inlier_ = ransac(keypoints_1,keypoints_2,matches, m, print_vals = False)
        Inliers_mat[i,m-1] = best_inlier_

In [ ]:
plt.figure()

plt.title('Convergence of average number of max inliers')
plt.plot(N_vec, np.mean(Inliers_mat, axis = 0))
plt.xlabel('Number of itterations N')
plt.ylabel('Average number of inliers after N itterations')
txt= r"Figure 2.2.1: The average number of inliers after " + str(E_experiments) +" experiments with N $\in$ [" + str(N_vec[0]) + ", " + str(N_vec[-1])+ "] itterations, illustrating the convergence values of the max number of inliers"
plt.figtext(0.5, -0.02, txt, wrap=True, horizontalalignment='center', fontsize=15)
    
plt.show()

____

# Image Stitching (40pts)

In this practice, you will write a function that takes two images as input and stitch them together. The method described in the previous section will be used to stitch two images together by transforming one of them to the coordinate space of the other. You will work with supplied images *left.jpg* and *right.jpg*. The overall scheme can be summarized as follows:

1.   As in previous task you should first find the best transformation between input images.

2.   Then you should estimate the size of the stitched image.

3.   Finally, combine the *left.jpg* with the transformed *right.jpg* into one image.

## Question 3
### Question 3.1 
Create a function that takes an image pair as input, and return the stitched version.




In [ ]:
def stitchImages(img1, img2, N):
    """
    Given two input images, return the stitched image.
    Arguments:
    img1: the first image (in RGB)
    img2: the second image (in RGB)
    Returns: 
    The keypoint matchings between the two image
    """
    
    #1. Find the best transformation.
    
    # Find and match key points
    keypoints_1, keypoints_2, matches = keypoint_matching(img1,img2)
    
    # Extract 10 random matches to plot
    random_matches = [matches[i] for i in random.sample(range(0, len(matches)), 10)]

    # Now plot them. Hint: for generating the plot, you can use cv2.drawMatches()

    img3 = cv2.drawMatches(img1, keypoints_1, img2, keypoints_2, random_matches,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    plt.figure(figsize = (15,15))
    plt.imshow(img3),plt.show()
    cv2.imwrite('sift_keypoints_3_bus.jpg',img3)
    
    best_matrix = ransac(keypoints_1,keypoints_2,matches, N_iterations)
    
    # Apply transformation
    dst_forward = affine_transform(img1, best_matrix, warp = 'inverse')
    # Display the image
    plt.figure(figsize = (20,20))
    plt.subplot(121)
    plt.imshow(img1)
    plt.title('Original image')
    plt.subplot(122)
    plt.imshow(dst_forward)
    plt.title('Open CV warping')
    plt.show()
    
    M, N = img2.shape[:2]

    dst_forward[0:M, 0:N, :] = img2
    plt.figure(figsize = (15,15))
    plt.imshow(dst_forward); plt.axis('off')
    plt.show()
    
    
    
    # TODO: 2. Estimate the size of the stitched image.
    # Hint: Calculate the transformed coordinates of corners of the *right.jpg*

    # TODO: 3. Combine the *left.jpg* with the transformed *right.jpg* into one image.

    # ================
    # Your code here
    # ================




In [ ]:
img1_path = "left.jpg"
img2_path = "right.jpg"

# Load images
img1 = cv2.imread(img1_path)
img2 = cv2.imread(img2_path)

# Note: OpenCV uses BGR instead of RGB
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) 
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

# Stitch images
N_iterations = 50 # Select based on your previous findings.
stitchedImage = stitchImages(img1, img2, N_iterations)

#plt.imshow(stitchedImage)
#plt.axis('off')
#plt.show()
#plt.close()

### Question 3.2
Visualize the stitched image alongside with the image pair.

In [ ]:
# TODO: Visualize the stitched image alongside with the image pair.

# ================
# Your code here
# ================